In [4]:
import collections

class Solution:
    '''
    First Step: record the index of each character in each file, find out the character which contained in more than two files
    Second Step: if we could find the character that contained in more than two files, we could use them as the prefix, find out
    the next common character
    Thrid Step: continue second step until we couldn't find more
    e.g. file1 = BACEA file2 = EE file3 = BACFCE
    we first find out the common character are A,B,C,E
    then we could use use B to find BA until we find BAC
    '''
    def __init__(self, filename_list):
        self.d_filestring = {}
        #dictionary: use to store each binary file as a string
        # key is the file name, value is the string of the file
        for filename in filename_list:
            self.d_filestring[filename] = open(filename,"rb").read()

        self.len_longest_strand = 0 #record the longest strand
        self.filenames_longest_strand = None 
        ## dictionary， key is filename, value is the index of common string that contained in other file

    def get_index_of_characters(self, filestring):
        '''
        input: the binary file in string format
        output: defaultdict, key is the character in that file, value is the list of index about that character
        e.g. file_a = "ABACE" 
        if we wanna get the index of characters in file_a, simply call function get_index_of_characters("ABACE")
        and the output is {A:[0,2] B:[1] C:[3] E:[4]}
        '''
        index_of_characters = collections.defaultdict(list)
        for index, character in enumerate(filestring):
            index_of_characters[character].append(index)
        return index_of_characters

    def search_next_character(self, common_string_index_dict, length_common_string):
        '''
        search for the next common character contained in file list
        input: common_string_index_dict:the common_string_index_dict we genearted 
               length_common_string: length of common string we wanna search
        '''
        # if we find the longer length of common string, update the len_longest_strand and file_names
        if length_common_string > self.len_longest_strand:
            self.len_longest_strand = length_common_string
            self.filenames_longest_strand = common_string_index_dict
        all_match = False
        offset = 1
        # check if we fully match the next character
        while self.next_character_all_match(common_string_index_dict, length_common_string + offset - 1):
            all_match = True
            offset += 1
        if all_match: ## fully match: no change and just move the offset
            self.search_next_character(common_string_index_dict, length_common_string + offset - 1)
        else: ## partial match: find a subset of common_string_index_dict and candidate next_characters
            next_character_to_index = collections.defaultdict(lambda : collections.defaultdict(list))
            # dict of dict , {filename:{nextcharacter:[list of index]}}
            character_count_among_files = collections.defaultdict(list)
            # dictionary key is the next character we wanna find, value is the filename that contained that next character
            for filename in common_string_index_dict:
                character_set_in_file = set()
                index_list = common_string_index_dict[filename]
                for index in index_list:
                    if index + length_common_string >= len(self.d_filestring[filename]):# out of index, break 
                        break
                    # find our the next character and append 
                    next_character = self.d_filestring[filename][index + length_common_string]
                    next_character_to_index[filename][next_character].append(index)
                    character_set_in_file.add(next_character)
                for next_character in character_set_in_file:
                    character_count_among_files[next_character].append(filename)

            for next_character in character_count_among_files:
                # if we find more than two files contained that next character
                if len(character_count_among_files[next_character]) >= 2:
                    common_string_index_dict = {filename:next_character_to_index[filename][next_character] for filename in character_count_among_files[next_character]}
                    if not self.last_character_all_match(common_string_index_dict):
                        # if last character are not all the same, we will search for next common string
                        self.search_next_character(common_string_index_dict, length_common_string + 1)

    def next_character_all_match(self, common_string_index_dict, offset):
        '''
        to see if we have all the same last/next character 
        input: common_string_index_dict:the common_string_index_dict we genearted 
               offset:if offset = -1, then we search for last character
               if offsect >0 : then we search for next character
        return: boolean, True if we find the next character are all the same False otherwise
        '''
        character_set = set()
        #use to record how many different next character we have 
        for filename in common_string_index_dict:
            index_list = common_string_index_dict[filename]
            for index in index_list:
                if index + offset < 0 or index + offset >= len(self.d_filestring[filename]):
                    return False
                next_character = self.d_filestring[filename][index + offset]
                character_set.add(next_character)
        if len(character_set) == 1:
            return True
        return False

    def last_character_all_match(self, common_string_index_dict):
        # if we find our last character are all the same, return False otherwise return true
        # for example if we are search for 'abc' as our next common character
        # we don't need to search for 'bc' since 'bc' is included in 'abc'
        return self.next_character_all_match(common_string_index_dict, offset = -1)

    def search_common_string_in_multiple_files(self):
        char_inds = {}
        #dictionary key:filename value:list of index ofor each character in that file
        character_count_among_files = collections.defaultdict(list)
        #dictionary key:character  value: list of filename that contained the key
        for filename in self.d_filestring:
            filestring = self.d_filestring[filename]
            #find the index of each character in each file
            index_of_characters = self.get_index_of_characters(filestring)
            #index_of_characters is the dictionary, key is characters and value is the list of index
            char_inds[filename] = index_of_characters
            # store the current filename as the key, and value is the dict of list
            for character in index_of_characters.keys():
                # process the character in current file to see if there's two or more files contained that characters
                character_count_among_files[character].append(filename)
        for character in character_count_among_files:
            # iterate each character
            # # if the character appears in more than two files
            # we could use it as the prefix to find the next common string with length of 2
            if len(character_count_among_files[character]) >= 2:
                #generate a dict that key is file name, and value is the index of character that also contained in other files
                common_string_index_dict = {filename:char_inds[filename][character] for filename in character_count_among_files[character]}
                #if we find the last character are not all the same, search for next common character
                if not self.last_character_all_match(common_string_index_dict):
                    self.search_next_character(common_string_index_dict, 1)

# Unit Testing

In [20]:
import unittest

class TestSearchCommonString(unittest.TestCase):

    def test_example(self):
        with open("testfile0", "wb") as fp:
            fp.write(b"ABACE")
        with open("testfile1", "wb") as fp:
            fp.write(b"EE")
        with open("testfile2", "wb") as fp:
            fp.write(b"BACFCE")
        
        solution = Solution(["testfile0", "testfile1", "testfile2"])
        solution.search_common_string_in_multiple_files()
        self.assertEqual(solution.len_longest_strand, 3)
        self.assertEqual(len(solution.filenames_longest_strand), 2)
        self.assertEqual("testfile0" in solution.filenames_longest_strand, True)
        self.assertEqual("testfile2" in solution.filenames_longest_strand, True)
        self.assertEqual(solution.filenames_longest_strand["testfile0"], [1])
        self.assertEqual(solution.filenames_longest_strand["testfile2"], [0])
        
    def test_example2(self):
        with open("testfile0", "wb") as fp:
            fp.write(b"CBCBD")
        with open("testfile1", "wb") as fp:
            fp.write(b"BCD")
        with open("testfile2", "wb") as fp:
            fp.write(b"BCBDC")
        
        solution = Solution(["testfile0", "testfile1", "testfile2"])
        solution.search_common_string_in_multiple_files()
        self.assertEqual(solution.len_longest_strand, 4)
        self.assertEqual(len(solution.filenames_longest_strand), 2)
        self.assertEqual("testfile0" in solution.filenames_longest_strand, True)
        self.assertEqual("testfile2" in solution.filenames_longest_strand, True)
        self.assertEqual(solution.filenames_longest_strand["testfile0"], [1])
        self.assertEqual(solution.filenames_longest_strand["testfile2"], [0])
        
    def test_example3(self):
        with open("testfile0", "wb") as fp:
            fp.write(b"")
        with open("testfile1", "wb") as fp:
            fp.write(b"AAA")
        with open("testfile2", "wb") as fp:
            fp.write(b"BBB")
        with open("testfile3", "wb") as fp:
            fp.write(b"CCC")
        with open("testfile4", "wb") as fp:
            fp.write(b"DDD")
        with open("testfile5", "wb") as fp:
            fp.write(b"EEE")
        
        solution = Solution(["testfile0", "testfile1", "testfile2", "testfile3", "testfile4", "testfile5"])
        solution.search_common_string_in_multiple_files()
        self.assertEqual(solution.len_longest_strand, 0)
        self.assertEqual(solution.filenames_longest_strand, None)
    
    def test_example4(self):
        with open("testfile0", "wb") as fp:
            fp.write(b"CBCBD")
            
        solution = Solution(["testfile0"])
        solution.search_common_string_in_multiple_files()
        self.assertEqual(solution.len_longest_strand, 0)
        self.assertEqual(solution.filenames_longest_strand, None)

In [21]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_example (__main__.TestSearchCommonString) ... C:\Users\yaozl\AppData\Local\Temp/ipykernel_70452/847881318.py:10: ResourceWarning: unclosed file <_io.BufferedReader name='testfile0'>
  self.d_filestring[filename] = open(filename,"rb").read()
C:\Users\yaozl\AppData\Local\Temp/ipykernel_70452/847881318.py:10: ResourceWarning: unclosed file <_io.BufferedReader name='testfile1'>
  self.d_filestring[filename] = open(filename,"rb").read()
C:\Users\yaozl\AppData\Local\Temp/ipykernel_70452/847881318.py:10: ResourceWarning: unclosed file <_io.BufferedReader name='testfile2'>
  self.d_filestring[filename] = open(filename,"rb").read()
ok
test_example2 (__main__.TestSearchCommonString) ... ok
test_example3 (__main__.TestSearchCommonString) ... C:\Users\yaozl\AppData\Local\Temp/ipykernel_70452/847881318.py:10: ResourceWarning: unclosed file <_io.BufferedReader name='testfile3'>
  self.d_filestring[filename] = open(filename,"rb").read()
C:\Users\yaozl\AppData\Local\Temp/ipykernel_70452/847881318

In [5]:
solution = Solution(["sample.%d" % i for i in range(1,11)])
solution.search_common_string_in_multiple_files()
print(solution.len_longest_strand)
print(solution.filenames_longest_strand)

27648
{'sample.2': [3072], 'sample.3': [17408]}
